In [1]:
from pycalphad import Database, Model, variables as v

dbf = Database('alfe_sei.TDB')

mod = Model(dbf, ['AL', 'FE', 'VA'], 'B2_BCC')

In [2]:
from collections import OrderedDict
from pycalphad.core.constraints import build_constraints
res = build_constraints(mod, [v.P, v.T] + mod.site_fractions, OrderedDict([(v.P, 101325), (v.T, 300), (v.N, 1), (v.X('FE'), 0.2)]))

In [3]:
import numpy as np
res.multiphase_jac.kernel(np.array([[101325, 300, 0.5, 0.5, 0.5, 0.5, 1, 1]]), np.array([]))

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ],
       [ 0.  ,  0.  , -0.25,  0.25, -0.25,  0.25,  0.  ,  0.5 ]])

In [4]:
from pycalphad import equilibrium

res = equilibrium(dbf, ['AL', 'FE', 'VA'], 'LIQUID', {v.P: 101325, v.T: 300, v.N: 1, v.X('AL'): 0.2})

mass_gradient [[ 0.8 -0.8]
 [-0.2  0.2]
 [ 0.2  0.8]]
jacobian [[ 1.   1.   0. ]
 [ 0.   0.   1. ]
 [ 0.8 -0.2  0.2]]
gradient [-61300.26030265   2412.49457566  -8168.06986545]
test_jacobian [[ 1.   1.   0. ]
 [ 0.   0.   1. ]
 [-0.8  0.2  0.8]]
guess_potentials (array([-10330.0564    , -59138.2737681 ,  63712.75487831]), array([], dtype=float64), 3, array([ 1.57731831,  1.2558771 ,  0.50481647]))
chemical_potentials [  4574.48111021 -63712.75487831]
test [ 54629.78879082 -13657.4471977  -50055.30768061]
mass_gradient [[ 0.8        -0.59146541]
 [-0.2         0.14786635]
 [ 0.8        -0.20853459]
 [-0.2         0.05213365]
 [ 0.2         0.8       ]
 [ 0.2         0.8       ]]
jacobian [[ 1.          1.          0.          0.          0.          0.        ]
 [ 0.          0.          1.          1.          0.          0.        ]
 [ 0.          0.          0.          0.          1.          1.        ]
 [ 0.59146541 -0.14786635  0.20853459 -0.05213365  0.2         0.2       ]]
gra

In [5]:
res

<xarray.Dataset>
Dimensions:       (N: 1, P: 1, T: 1, X_AL: 1, component: 2, internal_dof: 2, vertex: 2)
Coordinates:
  * X_AL          (X_AL) float64 0.2
  * component     (component) <U2 'AL' 'FE'
  * N             (N) float64 1.0
  * P             (P) float64 1.013e+05
  * T             (T) float64 300.0
  * vertex        (vertex) int64 0 1
  * internal_dof  (internal_dof) int64 0 1
Data variables:
    Phase         (N, P, T, X_AL, vertex) <U6 'LIQUID' ''
    Y             (N, P, T, X_AL, vertex, internal_dof) float64 0.2 0.8 nan nan
    MU            (N, P, T, X_AL, component) float64 4.574e+03 -6.371e+04
    X             (N, P, T, X_AL, vertex, component) float64 0.2 0.8 nan nan
    GM            (N, P, T, X_AL) float64 -8.168e+03
    NP            (N, P, T, X_AL, vertex) float64 1.0 nan
Attributes:
    engine: pycalphad 0.7+14.ge7d5773.dirty
    created: 2018-04-20T22:13:39.053636

In [6]:
mod.moles('FE').diff(v.Y('B2_BCC', 0, 'FE'))

-0.5*(0.5*B2_BCC0FE + 0.5*B2_BCC1FE)/(0.5*B2_BCC0AL + 0.5*B2_BCC0FE + 0.5*B2_BCC1AL + 0.5*B2_BCC1FE)**2 + 0.5/(0.5*B2_BCC0AL + 0.5*B2_BCC0FE + 0.5*B2_BCC1AL + 0.5*B2_BCC1FE)

In [7]:
from pycalphad.core.constraints import get_multiphase_constraint_rhs
get_multiphase_constraint_rhs(OrderedDict([(v.P, 101325), (v.T, 300), (v.N, 1), (v.X('FE'), 0.2)]))

[101325.0, 300.0, 1.0, 0.2]